<a href="https://colab.research.google.com/github/y4c6/master_thesis/blob/main/EJMR_doubleml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Using the data `ejmr_20_token.pkl` from `EJMR_LDA_W2V.ipynb`, i do LASSO and DOUBLEML in this file.

## import packages

In [2]:
!pip install -U DoubleML

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 KB 4.1 MB/s eta 0:00:00


In [4]:
import doubleml

In [23]:
import pandas as pd
import numpy as np

In [7]:
from google.colab import drive
# directory
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [11]:
df = pd.read_pickle("/content/gdrive/MyDrive/Thesis_Data&Result/ejmr_20_token.pkl") 
df.head()

,topic,posts,Target,y,text_tokenized
2,https://www.econjobrumors.com/topic/russia-chi...,[Is this the genius outcome the West was looki...,china,1,"[genius, outcome, west, looking, russian, fina..."
23,https://www.econjobrumors.com/topic/are-more-t...,"[Not even accounting for Indians., All the qua...",asia,0,"[even, accounting, indians, quant, clubs, univ..."
44,https://www.econjobrumors.com/topic/reminder-c...,"[Never forget. Never forgive., .]",china,1,"[never, forget, never, forgive]"
69,https://www.econjobrumors.com/topic/chinese-br...,[Before the Olympics and during the Olympics a...,china,1,"[olympics, olympics, news, whether, would, war..."
79,https://www.econjobrumors.com/topic/us-uses-uk...,"[2014 Ukrainian ""revolution"" is just another 9...",asia,0,"[ukrainian, revolution, another, incident]"


In [12]:
data = df[['y', 'text_tokenized']]
data.head()

,y,text_tokenized
2,1,"[genius, outcome, west, looking, russian, fina..."
23,0,"[even, accounting, indians, quant, clubs, univ..."
44,1,"[never, forget, never, forgive]"
69,1,"[olympics, olympics, news, whether, would, war..."
79,0,"[ukrainian, revolution, another, incident]"


In [13]:
from sklearn.feature_extraction.text import CountVectorizer

# I've have the tokens, so i pass dummy function in cv.
def dummy(doc):
    return doc

# Create a matrix using CountVectorizer
cv = CountVectorizer(preprocessor = dummy, 
            tokenizer = dummy, 
            max_features = 10000)

In [27]:
data_matrix = cv.fit_transform(df['text_tokenized'])

data_matrix_df = pd.DataFrame(data_matrix.toarray(), columns = cv.get_feature_names())

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


## LASSO

In [17]:
from sklearn.linear_model import Lasso

lasso = Lasso(alpha=0.1)

In [28]:
lasso.fit(data_matrix_df, df.y)

Lasso(alpha=0.1)

In [29]:
print(lasso.coef_)

[-0. -0.  0. ...  0.  0.  0.]


In [34]:
len(lasso.coef_[np.nonzero(lasso.coef_)])
print(data_matrix_df.columns[np.nonzero(lasso.coef_)])

Index(['asian', 'china', 'chinese', 'japan', 'kimchi', 'korean', 'like',
       'taiwan'],
      dtype='object')


Only 8 words are non zero.
They are `['asian', 'china', 'chinese', 'japan', 'kimchi', 'korean', 'like', 'taiwan']`.

But most of these words are targets. I think I have to remove them.  

In [40]:
asia_target = ['asian', 'asia', 'korea', 'korean', 'japan', 'japanese', 'taiwan', 'taiwanese', 'east', 'hongkong']
china_target = ['china', 'chinese']

In [46]:
# Defining all the conditions inside a function
def remove_target(x):
  return [word for word in x if word not in (asia_target + china_target)]

# Applying the conditions
data['text_tokenized_cleaned'] = data['text_tokenized'].apply(remove_target)
data.head(3)

<ipython-input-46-16949c5afc94>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text_tokenized_cleaned'] = data['text_tokenized'].apply(remove_target)


,y,text_tokenized,text_tokenized_cleaned
2,1,"[genius, outcome, west, looking, russian, fina...","[genius, outcome, west, looking, russian, fina..."
23,0,"[even, accounting, indians, quant, clubs, univ...","[even, accounting, indians, quant, clubs, univ..."
44,1,"[never, forget, never, forgive]","[never, forget, never, forgive]"


In [61]:
type(data.text_tokenized)

pandas.core.series.Series

In [81]:
for index, row in data.head(5).iterrows():
    print(len(row['text_tokenized']) - len(row['text_tokenized_cleaned']))

18
7
0
6
0


In [83]:
data_matrix_cleaned = cv.fit_transform(data['text_tokenized_cleaned'])

data_matrix_df_cleaned = pd.DataFrame(data_matrix_cleaned.toarray(), columns = cv.get_feature_names())

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [84]:
lasso.fit(data_matrix_df_cleaned, df.y)

Lasso(alpha=0.1)

In [85]:
len(lasso.coef_[np.nonzero(lasso.coef_)])
print(data_matrix_df_cleaned.columns[np.nonzero(lasso.coef_)])

Index(['asians', 'ccp', 'us', 'w'], dtype='object')


After I cleaned up the target words, there're only words are nonzero.
So, lemmatization IS NEEDED. And wtf is `'w'`?